## Follow these steps before continuing
1. Run get_deposit_withdrawal_history.ipynb
2. Switch the kernel for this notebook to get_deposit_withdrawal_history.ipynb

## If this is not your first time running the notebook:
3. Add new trading pairs to `data/trading_pairs.txt` if they are not in the file already

In [ ]:
import sys
sys.path.append("../scripts")
from binance.client import Client
import UTC_time_converter as UTC
import numpy as np
import pickle as pk
import time
import pytz
import re
import binance.exceptions as Exceptions
from ratelimiter import RateLimiter
import pandas as pd

## Initialize client and check connection status

In [ ]:
#Initialize client
api_key, api_secret = open('../data/client_info.txt').read().split('\n')
client = Client(api_key, api_secret)

In [ ]:
#Ping the server
client.ping()

## Get updated API rate limit
Request weight of `client.get_my_trades(symbol)` = 5 per request.

In [ ]:
#Get request weight per minute
API_RATELIMIT = client.get_exchange_info()['rateLimits'][0]['limit']
print(client.get_exchange_info()['rateLimits'][0])
print(API_RATELIMIT)

In [ ]:
#define rate limiter
rate_limiter = RateLimiter(max_calls=0.9*API_RATELIMIT/5, period=60) #period is in units of seconds

#check my math. If it doesn't print True, I can't do math
if 0.9*API_RATELIMIT/5 < API_RATELIMIT : print(True)

## Initialize locally saved trade_history

**Run this everytime you have traded with a new trading pair**

**HTTP 4XX** return codes are used for malformed requests; the issue is on the sender's side.

**HTTP 403** return code is used when the WAF Limit (Web Application Firewall) has been violated.

**HTTP 429** return code is used when breaking a request rate limit.

**HTTP 418** return code is used when an IP has been auto-banned for continuing to send requests after receiving 429 codes.


### Get list of all existing trading pairs on Binance

In [ ]:
trading_pairs = [trading_pair['symbol'] for trading_pair in client.get_exchange_info()['symbols']]
print(trading_pairs[:10])

`trading_pairs` is a list of all valid (and previously valid but not valid anymore) trading pairs

In [ ]:
#test with known valid trading pair
if any(re.match('ETHBTC', i) for i in trading_pairs): print('TRUE')
#find trading pair that used to be valid
if any(re.match('BCHSVBTC', i) for i in trading_pairs): print('TRUE')

In [ ]:
t0 = time.time()
trade_history = []
for i in trading_pairs:
    
    #RATE LIMITER
    with rate_limiter:
        
        #Fetch trade history for each trading_pair
        try: trade_history += client.get_my_trades(symbol=i)
            
        #Address error messages
        except Exceptions.BinanceAPIException as e: 
            print(e.status_code, e.message)
            
            #These error messages happen when exceeding rate limit
            if int(e.status_code) in [403, 429, 418]: 
                print("Please lower rate limit")
                break
            
            #Error 400: can't find trading_pair
            if int(e.status_code) == 400: 
                pass
        
t1 = time.time()

In [ ]:
print(t1-t0, "s")

####  SAVE THIS PRECIOUS INFORMATION AND BACK IT UP

In [ ]:
trade_history = pd.DataFrame(trade_history)
with open('../data/trade_history.pk', 'wb') as f: pk.dump(trade_history, f) 
with open('../data/trade_history_copy.pk', 'wb') as f: pk.dump(trade_history, f) 

#### Initialize `trading_pairs.txt`
`trading_pairs.txt` contains the list of trading pairs of interest

In [ ]:
with open('../data/trading_pairs.txt', 'w') as f: 
    for i in np.unique(list(trade_history.symbol)):
        f.write(str(i)+'\n')

## Query only trading pairs that you have traded before

In [ ]:
with open('../data/trading_pairs.txt', 'r') as f: 
    trading_pairs = [line.rstrip('\n') for line in f]
    
t0 = time.time()
trade_history = []
for i in trading_pairs:
    
    #RATE LIMITER
    with rate_limiter:
        
        #Fetch trade history for each trading_pair
        try: trade_history += client.get_my_trades(symbol=i)
            
        #Address error messages
        except Exceptions.BinanceAPIException as e: 
            print(e.status_code, e.message)
            
            #These error messages happen when exceeding rate limit
            if int(e.status_code) in [403, 429, 418]: 
                print("Please lower rate limit")
                break
            
            #Error 400: can't find trading_pair
            if int(e.status_code) == 400: 
                pass

#Convert to dataframe and SAVE
trade_history = pd.DataFrame(trade_history)
with open('../data/trade_history.pk', 'wb') as f: pk.dump(trade_history, f) 
    
t1 = time.time()

In [ ]:
print(t1-t0, "s")